`What is a subject?` $\implies$ Any Python function, method, class or block of code. 

We need to create the Agent that will decide the tool to use based on a query. There will be two agents: 
1. **Simple vs Complex Agent**. This agent will classify the query of the user into one of these categories: 

- Simple:  Zero or nne subject present in the question. *Example: How does the function x work?*
- Complex: More than one subject.  *Example: What are the differences between Class A and Class B?*

- Tool returned: **`SimpleRetriever`**


2. **General vs Particular Agent**. This agent will the output from the first agent only if the question type was: **`Simple`** and classify the question into one of two categories: 

- Particular: The question involves only the subject. *Example: How does the function x work?*
- General: The question is formulated in such a way that the question does not have to do only with the subject. *Will my changes break anything?*

- Tool returned: **`SimpleRetriever`** or **`GeneralRetriever`**

In [1]:
from app.agent.multi_agent import MultiAgent
from app.agent.agent import ContextTypeAgent, QuestionTypeAgent
from app.prompts.prompts import SIMPLE_VS_COMPLEX, GENERAL_VS_PARTICULAR_CONTEXT
from app.prompts.prompt import Prompt

import asyncio
import nest_asyncio
nest_asyncio.apply()

# simple vs complex -> 0-1 vs > 1 subjects
simple_vs_complex_prompt = Prompt(prompt=SIMPLE_VS_COMPLEX)
simple_vs_complex_agent = QuestionTypeAgent(
    instruction=simple_vs_complex_prompt
)

# general vs particular
general_vs_particular_prompt = Prompt(GENERAL_VS_PARTICULAR_CONTEXT)
general_vs_particular_agent = ContextTypeAgent(
    instruction=general_vs_particular_prompt
    )

multi_agent = MultiAgent(agents=[
    simple_vs_complex_agent, 
    general_vs_particular_agent
])

/Users/Jaime/Desktop/Project-Ideas/python-gpt/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Classification Testing

**We are going to test how the agent classify the questions and what tools does it choose to answer the question**

In [2]:
from app.database.base import get_db
from sqlalchemy.orm import Session

db = next(get_db())

In [3]:
from app.retrievers.general_retriever import GeneralRetriever
from app.retrievers.similarity_retriever import SimilarityRetriever


async def tool_pipeline(agent: MultiAgent, query: str, db: Session=db):
    tool, output = await agent.pipeline(query=query)
    tool: GeneralRetriever | SimilarityRetriever = tool(db=db)
    return tool.query_database(query=query, subjects=output.subject)

### 1.1 General question retriever

**This one is a tricky one because we are specifying a line of code besides the function. That is not an issue because of how our retrievers are built. We'll look to see if can find the subject based on the function name, method name or class name fields of the NodeMetadata.node_metadata column. That is why Postgres is so powerfull!**

In [5]:
query = "What will happen If I remove the line 'os.remove(name_file)' of the function _create_node_relationships_file"
nodes, nodes_with_score = asyncio.run(tool_pipeline(agent=multi_agent, query=query, db=db))

Agent reasoning response: The subject is the function _create_node_relationships_file therefore the answer is simple because there is only one subject
Agent answer: simple 

Agent reasoning response: The question refers to what will happen when removing a line from the function, which is not about the subject itself, but about its effects, therefore the answer is general
Agent answer: general 

Tool decided by the agent: GeneralRetriever


/Users/Jaime/Desktop/Project-Ideas/python-gpt/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


	Exact match of subject: {'_create_node_relationships_file'} in the database. --> 1


In [6]:
# we print some of the nodes in which the function _create_node_relationships_file appears
for node in nodes:
    print(node.text[:300])

def _create_node_relationships_file(db: Session):
    
    fields = ('class_name', 'function_name', 'method_name')
    name_file = os.environ['NAMES_FILE']
    relationships_file = os.environ['RELATIONSHIPS_FILE']
    
    os.remove(name_file)
    
    if not os.path.exists(name_file):
        nodes
async def update_nodes_store(db: Session = Depends(get_db)):
    updated_files = _create_file_node(path=os.environ['USER_CODE_DIRECTORY'], db=db)
    _create_node_relationships_file(db=db)
    return {"py_files": updated_files}

async def upload_file_zip(file: UploadFile = File(...), db: Session = Depends(get_db)):

    extract_dir = os.environ['USER_CODE_DIRECTORY']
    if not os.path.exists(extract_dir):
        # shutil.rmtree(extract_dir)
        
        os.makedirs(extract_dir, exist_ok=True)

        with open(f"{ext


### 1.2 Particular Question retrieval

**Let's see one in which several retries have been needed to actually get a coherent answer: multiple subjects $\implies$ complex question whereas one subject $\implies$ simple.**

In [7]:
query = "Explain to me how the class NodePostProccesor is implemented"
nodes, _ = asyncio.run(tool_pipeline(agent=multi_agent, query=query, db=db))

Agent reasoning response: The subject is the class NodePostProccesor therefore the answer is complex because there is one class.
Agent answer: complex 

Not valid answer. Applying retries
	Retry agent response: {"question_type": "complex", "subject": ["NodePostProccesor"], "reasoning": "The subject is the class NodePostProccesor therefore the answer is complex because there is one subject"}
	Retry agent response: {"question_type": "complex", "subject": ["NodePostProccesor"], "reasoning": "The subject is the class NodePostProccesor therefore the answer is complex because there is only one subject, which is a class"}
	Retry agent response: {"question_type": "complex", "subject": ["NodePostProccesor"], "reasoning": "The subject is the class NodePostProccesor therefore the answer is complex because there is only one subject which is a class"}
	Retry agent response: {"question_type": "simple", "subject": ["NodePostProccesor"], "reasoning": "The subject is the class NodePostProccesor therefo

In [8]:
for node in nodes:
    print(node.text[:300])

class NodePostProccesor:
  
    def __init__(self, retrieved_nodes_score: List[NodeWithScore], db: Session, score_threshold: float=0.8, min_parent_nodes: int=2, min_file_nodes: int=2):
        
        self._retrieved_nodes_score = [node for node in retrieved_nodes_score if node.score > score_thresh


**As we can see, the output has modified the name of the subject: NodePostProcesor has changed to NodePostProcessor. This implies that we won't get a perfect match from the database, so similarity search will be performed. We still get the appropiate Node.**

In [9]:
# let's try again with the same question but removing the word class
# this time it gets a coherent answer faster
query = "Explain to me how NodePostProccesor is implemented"
nodes = asyncio.run(tool_pipeline(agent=multi_agent, query=query, db=db))

Agent reasoning response: The subject is NodePostProccesor class therefore the answer is complex because there is one subject.
Agent answer: complex 

Not valid answer. Applying retries
	Retry agent response: {"question_type": "simple", "subject": ["NodePostProcessor"], "reasoning": "The subject is NodePostProcessor, therefore the answer is simple because there is only one subject"}
Agent reasoning response: The question is regarding the implementation of NodePostProcessor itself, therefore the answer is particular
Agent answer: particular 

Tool decided by the agent: SimilarityRetriever


/Users/Jaime/Desktop/Project-Ideas/python-gpt/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Original Query: Explain to me how NodePostProccesor is implemented
	New query to look with subject: NodePostProcessor -->  explain to me how nodepostproccesor is implemented


In [10]:
for node in nodes[0]:
    print(node.text[:300])

class NodePostProccesor:
  
    def __init__(self, retrieved_nodes_score: List[NodeWithScore], db: Session, score_threshold: float=0.8, min_parent_nodes: int=2, min_file_nodes: int=2):
        
        self._retrieved_nodes_score = [node for node in retrieved_nodes_score if node.score > score_thresh


### 1.3 Complex question retrieval

**Now we want our Agent to identify multiple subjects.**

In [11]:
query = "How does the function _create_file_node depend on the function _create_nodes_of_file?"
nodes, _ = asyncio.run(tool_pipeline(agent=multi_agent, query=query, db=db))

Agent reasoning response: The subjects are: _create_file_node function and _create_nodes_of_file function, therefore the answer is complex because there are more than one subject.
Agent answer: complex 

Tool decided by the agent: SimilarityRetriever


/Users/Jaime/Desktop/Project-Ideas/python-gpt/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Original Query: How does the function _create_file_node depend on the function _create_nodes_of_file?
	Exact match of subject: _create_file_node in the database. --> 1
	Exact match of subject: _create_nodes_of_file in the database. --> 1


In [12]:
for node in nodes:
    print(node.text[:300])

def _create_file_node(path: str, db: Session):
    updated_files = []
    for root, _, files in os.walk(path):
        for file in files:
            lines = open(os.path.join(root, file), "r").readlines()
            text = "".join(lines)
            hash = calculate_hash(text)
            file = F
def _create_nodes_of_file(path: str, db: Session, file_id: str):
    files_structure_folder = os.environ['FILES_STRUCTURE_FOLDER']
    os.makedirs(files_structure_folder, exist_ok=True)
    
    if path.startswith("."): path = path[2:]
    elif path.startswith(".."): path = path[3:]
    
    file_na


# 2. Question-Answer with agents

**Now that we've seen that our agent classify properly some questions, let's actually find an answer.**

In [4]:
from app.agent.llama_client import LlamaClient
from app.printer import Printer
from app.prompts.prompts import PROMPT_TO_ANSWER_QUESTIONS

printer = Printer()
query = "How does the function _create_file_node depend on the function _create_nodes_of_file?"

llm = LlamaClient()

def format_answer(answer: str, max_words: int) -> str:
    answer = answer.replace("\n\n", "\n")
    lines = answer.split("\n")
    processed_answer = " "
    for line in lines:
        words = line.split(" ")
        for k in range(0, len(words), max_words):
            processed_line = " ".join(words[k: k + max_words])
            processed_answer += "\n" + processed_line
    return processed_answer

async def query_pipeline(agent: MultiAgent, 
                         query: str, 
                         llm: LlamaClient, 
                         db: Session) -> str:
    
    tool, output = await agent.pipeline(query=query)
    tool: GeneralRetriever | SimilarityRetriever = tool(db=db)
    nodes, nodes_with_score, relationships = tool.query_database(query=query, subjects=output.subject)
    for node_with_score in nodes_with_score:
        printer.print_blue(f"Score: {node_with_score.score} for text: \n{node_with_score.node.text[:300]}\n")
    if relationships: 
        for relation, relation_nodes in relationships.items():
            for n, rel_node in enumerate(relation_nodes):
                printer.print_blue(f"\tRelationship {n+1} for node --> {relation}: \n{rel_node.text[:150]}\n")
    context = "\n".join([node.text for node in nodes])
    prompt = Prompt.format_prompt(prompt=PROMPT_TO_ANSWER_QUESTIONS, context=context, query=query)
    answer = await llm.acall(query=prompt)
    return answer

In [8]:
answer = asyncio.run(
    query_pipeline(agent=multi_agent, 
                   query=query, 
                   llm=llm, 
                   db=db)
)

Agent reasoning response: The subjects are: _create_file_node function and _create_nodes_of_file function, therefore the answer is complex because there are more than one subject.
Agent answer: complex 

Tool decided by the agent: SimilarityRetriever


/Users/Jaime/Desktop/Project-Ideas/python-gpt/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Original Query: How does the function _create_file_node depend on the function _create_nodes_of_file?
	Exact match of subject: _create_nodes_of_file in the database. --> 1
	Exact match of subject: _create_file_node in the database. --> 1
Score: 1 for text: 
def _create_nodes_of_file(path: str, db: Session, file_id: str):
    files_structure_folder = os.environ['FILES_STRUCTURE_FOLDER']
    os.makedirs(files_structure_folder, exist_ok=True)
    
    if path.startswith("."): path = path[2:]
    elif path.startswith(".."): path = path[3:]
    
    file_na

Score: 1 for text: 
def _create_file_node(path: str, db: Session):
    updated_files = []
    for root, _, files in os.walk(path):
        for file in files:
            lines = open(os.path.join(root, file), "r").readlines()
            text = "".join(lines)
            hash = calculate_hash(text)
            file = F



In [9]:
formated_answer = format_answer(answer=answer, max_words=15)
print(formated_answer)

 
The function `_create_file_node` depends on the function `_create_nodes_of_file` in that it calls this function internally.
In particular, the line `db.add(file)` is followed by `_create_nodes_of_file(path=os.path.join(file.path), db=db, file_id=file.id)`, which suggests that once
a new file node is created, all nodes of that file need to be processed
and created using the `_create_nodes_of_file` function.
Therefore, the order in which these two functions are called implies that `_create_nodes_of_file` is used
to process the contents of each file.


In [38]:
query = "If I change the parameter depth of the method: __retrieve_relationship_nodes what effect will that have?"
answer = asyncio.run(
    query_pipeline(agent=multi_agent, 
                   query=query, 
                   llm=llm, 
                   db=db)
)

Agent reasoning response: The subject is the __retrieve_relationship_nodes method, therefore the answer is simple since there is only one subject
Agent answer: simple 

Agent reasoning response: The question involves the effect, which is different than the subject, therefore the answer is general.
Agent answer: general 

Tool decided by the agent: GeneralRetriever


/Users/Jaime/Desktop/Project-Ideas/python-gpt/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


	Exact match of subject: {'__retrieve_relationship_nodes'} in the database. --> 1
Score: 1 for text: 
    def __retrieve_relationship_nodes(self, base_id: str, node: Node, depth: int):
        if base_id == str(node.id) or depth == 0: 
            return [node.id]
        relations = []
        node_relationships = node.node_relationships
        if not node_relationships or not len(node_relationshi



In [40]:
formated_answer = format_answer(answer=answer, max_words=15)
print(formated_answer)

 
A great question!
The `depth` parameter in the `__retrieve_relationship_nodes` method controls how many levels of relationships are retrieved.
Let's look at the code:
```python
def __retrieve_relationship_nodes(self, base_id: str, node: Node, depth: int):
    ...
    if depth == 0:
        return [node.id]
    ...
    relations.extend(self.__retrieve_relationship_nodes(base_id=base_id, node=node_, depth=depth-1))
```
As you can see, the method calls itself recursively with a decreasing `depth` value until
it reaches `depth == 0`. When `depth` is 0, it simply returns a list containing
the ID of the current node.
If you change the `depth` parameter, you'll affect how many levels of relationships are retrieved:
* If `depth` is increased (e.g., from 2 to 3), more levels of relationships will
be retrieved.
* If `depth` is decreased (e.g., from 2 to 1), fewer levels of relationships will
be retrieved.
For example, if you call the method with `depth=2`, it will retrieve all relationships at

This is the same question, but know another subject is involved to confuse the Agent. The *NodePostProccesor* class is the parent class of the method *__retrieve_relationships_nodes*. 

In [43]:
query = "If I change the parameter depth of the method: __retrieve_relationship_nodes of the class NodePostProccesor what effect will that have?"
answer = asyncio.run(
    query_pipeline(agent=multi_agent, 
                   query=query, 
                   llm=llm, 
                   db=db)
)

Agent reasoning response: The subjects are: NodePostProcessor class and __retrieve_relationship_nodes method, therefore the answer is complex because there are more than one subject.
Agent answer: complex 

Tool decided by the agent: SimilarityRetriever


/Users/Jaime/Desktop/Project-Ideas/python-gpt/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Original Query: If I change the parameter depht of the method: __retrieve_relationship_nodes of the class NodePostProccesor what effect will that have?
	New query to look with subject: NodePostProcessor -->  if i change the parameter depht of the method: __retrieve_relationship_nodes of the class nodepostproccesor what effect will that have?
	Exact match of subject: __retrieve_relationship_nodes in the database. --> 1
Score: 0.760384105455402 for text: 
class NodePostProccesor:
  
    def __init__(self, retrieved_nodes_score: List[NodeWithScore], db: Session, score_threshold: float=0.8, min_parent_nodes: int=2, min_file_nodes: int=2):
        
        self._retrieved_nodes_score = [node for node in retrieved_nodes_score if node.score > score_thresh

Score: 1 for text: 
    def __retrieve_relationship_nodes(self, base_id: str, node: Node, depth: int):
        if base_id == str(node.id) or depth == 0: 
            return [node.id]
        relations = []
        node_relationships = node.

**VERY INTERESTING**. 


The *NodePostProcessor* has been corrected to: *NodePostProccessor*, so there is no match in subjects! Nevertheless, we have obtained the *NodePostProccesor* node thanks to similarity search, which is great!

In [44]:
formated_answer = format_answer(answer=answer, max_words=15)
print(formated_answer)

 
Let's dive into the code and see how the `depth` parameter affects the behavior of
the `__retrieve_relationship_nodes` method.
Here's the relevant code:
```python
def __retrieve_relationship_nodes(self, base_id: str, node: Node, depth: int):
    if base_id == str(node.id) or depth == 0:
        return [node.id]
    relations = []
    node_relationships = node.node_relationships
    if not node_relationships or not len(node_relationships):
        return [node.id]
    for id, _ in node_relationships.items():
        node_ = self._db.get(Node, id)
        relations.extend(self.__retrieve_relationship_nodes(base_id=base_id, node=node_, depth=depth-1))
    return relations
```
The `depth` parameter controls how many levels of relationships are recursively traversed. Here's what happens
when you change the value of `depth`:
* If `depth` is 0, the method will only return the current node's ID (`node.id`).
No recursive traversal will occur.
* If `depth` is greater than 0, the method will re

In [6]:
query = "What will happen in the repository If I remove the line 'os.remove(name_file)' of the function _create_node_relationships_file"
answer = asyncio.run(
    query_pipeline(agent=multi_agent, 
                   query=query, 
                   llm=llm, 
                   db=db)
)

Agent reasoning response: The subject is the function _create_node_relationships_file therefore the answer is complex because there is one subject.
Agent answer: complex 

Not valid answer. Applying retries
	Retry agent response: {"question_type": "complex", "subject": ["_create_node_relationships_file"], "reasoning": "The subject is _create_node_relationships_file function therefore there is only one subject, but the question involves the repository, so it's complex"}
	Retry agent response: {"question_type": "complex", "subject": ["_create_node_relationships_file"], "reasoning": "The subject is the function _create_node_relationships_file therefore the answer is complex because there is one function involved"}
	Retry agent response: {"question_type": "simple", "subject": ["_create_node_relationships_file"], "reasoning": "The subject is the function _create_node_relationships_file, therefore the answer is simple because there is only one subject."}
Agent reasoning response: The questio

/Users/Jaime/Desktop/Project-Ideas/python-gpt/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


	Exact match of subject: {'_create_node_relationships_file'} in the database. --> 1
Score: 1 for text: 
def _create_node_relationships_file(db: Session):
    
    fields = ('class_name', 'function_name', 'method_name')
    name_file = os.environ['NAMES_FILE']
    relationships_file = os.environ['RELATIONSHIPS_FILE']
    
    os.remove(name_file)
    
    if not os.path.exists(name_file):
        nodes



In [7]:
formated_answer = format_answer(answer=answer, max_words=15)
print(formated_answer)

 
Let's dive into what happens if we remove the `os.remove(name_file)` line from the `_create_node_relationships_file` function.
The original code removes the `name_file` before creating a new file with the same name.
The `name_file` is used to store node metadata information. If we remove this line, the
existing `name_file` will not be deleted, and the new contents will be appended to it
instead of overwriting the previous contents.
Here's an example of what might happen:
1. The function creates a new file with the same name (`name_file`) as before.
2. It then tries to read from this file and write to it, which would
result in appending new data to the end of the existing file instead of overwriting
it.
This might cause issues if there are existing node metadata entries that need to be
updated or deleted. The function will continue to append new data without clearing out any
previous contents.
To summarize: If we remove `os.remove(name_file)`, the existing `name_file` will not be dele

# 3. EDGE CASES

Let's see how the retriever and the answer will be in case we are not carefull with the query. 

In [5]:
query = "How does the method check_relationships_of_retrieved_nodes work?"
answer = asyncio.run(
    query_pipeline(agent=multi_agent, 
                   query=query, 
                   llm=llm, 
                   db=db)
)

Agent reasoning response: The subject is the method check_relationships_of_retrieved_nodes therefore the answer is simple because there is only one subject
Agent answer: simple 

Agent reasoning response: The question is regarding the method check_relationships_of_retrieved_nodes itself, therefore the answer is particular
Agent answer: particular 

Tool decided by the agent: SimilarityRetriever


/Users/Jaime/Desktop/Project-Ideas/python-gpt/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Original Query: How does the method check_relationships_of_retrieved_nodes work?
	New query to look with subject: check_relationships_of_retrieved_nodes -->  How does the method check_relationships_of_retrieved_nodes work?
Relationships of node retrieve: 4
Score: 0.8793947244412036 for text: 
    def _check_relationships_of_retrieved_nodes(self, depth: int) -> List[str]:
        relations = []
        for node in self._retrieved_nodes:
            id = node.id
            node_relationships = self._db.get(Node, id).node_relationships
            if not node_relationships or not len(node_

	Relationship 1 for node --> 7f5b16db-5362-456e-8bc7-3273c1b96559:     def __retrieve_relationship_nodes(self, base_id: str, node: Node, depth: int):
        if base_id == str(node.id) or depth == 0: 
            retu

	Relationship 2 for node --> 7f5b16db-5362-456e-8bc7-3273c1b96559:     def _check_relationships_of_retrieved_nodes(self, depth: int) -> List[str]:
        relations = []
        for nod

In [6]:
formated_answer = format_answer(answer=answer, max_words=15)
print(formated_answer)

 
I'm happy to explain how this method works!
The `check_relationships_of_retrieved_nodes` method takes an integer parameter called `depth`. It returns a list of strings.
Here's what it does:
1. It initializes an empty list called `relations`.
2. It iterates over each node in the `_retrieved_nodes` collection.
3. For each node, it retrieves the node's ID and its relationships (stored in `_db.get(Node,
id).node_relationships`).
4. If the node has no relationships or if the relationships list is empty, it
skips to the next iteration.
5. For each relationship, it retrieves the related node (`self._db.get(Node, node_relationship_id)`) and calls another method
(`__retrieve_relationship_nodes`) with the current node's ID, the related node, and the provided `depth`. This method
presumably returns a list of nodes or IDs.
6. It adds these returned relationships to the `relations` list using the `extend` method.
In essence, this method is traversing a graph of nodes and their relationships, expl

**As we can see, if we don't specify correctly the method, class or function, it performs a simple similarity search, so we may or not obtain the desired node.**

In [5]:
query = """
how do the methods: _check_common_parent_nodes, _check_relationships_of_retrieved_nodes, return_nodes_after_apply_threshold_filter and return_nodes_with_score_after_apply_threshold_filter work together 
to improve the result of the function query_vector_database?
"""

answer = asyncio.run(
    query_pipeline(agent=multi_agent, 
                   query=query, 
                   llm=llm, 
                   db=db)
)

Agent reasoning response: The subjects are the methods: _check_common_parent_nodes, _check_relationships_of_retrieved_nodes, return_nodes_after_apply_threshold_filter and return_nodes_with_score_after_apply_threshold_filter. These are multiple functions involved in the question, therefore it is complex.
Agent answer: complex 

Tool decided by the agent: SimilarityRetriever


/Users/Jaime/Desktop/Project-Ideas/python-gpt/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Original Query: 
how do the methods: _check_common_parent_nodes, _check_relationships_of_retrieved_nodes, return_nodes_after_apply_threshold_filter and return_nodes_with_score_after_apply_threshold_filter work together 
to improve the result of the function query_vector_database?

	Exact match of subject: return_nodes_after_apply_threshold_filter in the database. --> 1
	Exact match of subject: _check_common_parent_nodes in the database. --> 1
	Exact match of subject: _check_relationships_of_retrieved_nodes in the database. --> 1
	Exact match of subject: return_nodes_with_score_after_apply_threshold_filter in the database. --> 1
	Exact match of subject: query_vector_database in the database. --> 1
Score: 1 for text: 
    def return_nodes_after_apply_threshold_filter(self):
        return self._retrieved_nodes


Score: 1 for text: 
    def _check_common_parent_nodes(self) -> List[Tuple[str, int]]:
        
        parent_node_ids =  {}
        file_of_node_ids = {}
        
        for n

In [6]:
formated_answer = format_answer(answer=answer, max_words=15)
print(formated_answer)

 
I'd be happy to explain how these methods work together in the `query_vector_database` function.
The methods `_check_common_parent_nodes`, `_check_relationships_of_retrieved_nodes`, `return_nodes_after_apply_threshold_filter`, and `return_nodes_with_score_after_apply_threshold_filter` are all part of a class called `NodePostProccesor`.
This class is used to process the results of a vector database query, which returns
a list of nodes with their corresponding distances from the original query code.
Here's how these methods work together:
1. `_check_common_parent_nodes`: This method takes in the retrieved nodes (which are returned by the `query_vector_database`
function) and checks for common parent nodes and files. It does this by iterating over
each node, checking if it's a method node (based on its type), and then counting
the frequency of its parent nodes and file IDs. The method returns a list of
tuples, where each tuple contains a parent node ID or file ID along with its
frequenc

**This has been a lucky try! Let's see if we can repeat the result...**